In [35]:
import pandas as pd
from sklearn import ensemble, preprocessing, linear_model
import random
import glob

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [43]:
df = pd.concat([pd.read_csv(i) for i in glob.glob('/media/td/Samsung_T5/sports/nba/nn_architectures*')])
df.sort_values('accuracy', ascending = False).replace(False, 0).replace(True, 1).head(20)


,convolutional_kernel_size_1,convolutional_pool_size_1,pooling_algorithm_1,convolution_type_1,convolutional_layers_1,convolutional_filters_1,conv_layers_per_pooling_layer_1,conv_layers_per_pooling_layer_2,convolutional_kernel_size_2,convolutional_pool_size_2,pooling_algorithm_2,convolution_type_2,convolutional_layers_2,convolutional_filters_2,recurrent_type,use_x1,use_x2,optimizer_algorithm,batchnorm,block_1_activations,block_2_activations,merge_activations,top_block_activations,block_1_dropout,block_2_dropout,merge_block_dropout,top_block_dropout,merge_block_width,block_2_width,top_block_width,recurrent_layers_width,block_1_dense_layers_width,merge_block_layers,block_2_layers,dense_top_layers,recurrent_layers,block_1_dense_layers,block_1_dropout_amount,block_2_dropout_amount,merge_block_dropout_amount,top_block_dropout_amount,input_shape1,input_shape2,history_length,timeseries_scaled,general_feature_encoding_size,rotate_history_data,num_of_params,model_training_time,accuracy
1,5,4,layers.MaxPooling1D,layers.SeparableConv1D,1,67,2,3,3,1,NaN,layers.LocallyConnected1D,2,36,layers.SimpleRNN,0,1,optimizers.Nadam(),0,softplus,softplus,relu,softplus,layers.GaussianDropout,NaN,layers.GaussianNoise,layers.GaussianNoise,184,79,55,103,165,1,1,2,2,3,0.754094,0.537411,0.555939,0.340520,"(4, 50)","(128,)",4.0,1.0,128.0,0.0,17727.0,2.400554,0.656692
4,8,1,layers.MaxPooling1D,layers.SeparableConv1D,1,38,2,3,5,4,layers.MaxPooling1D,layers.Conv1D,0,113,layers.SimpleRNN,1,0,optimizers.Adamax(),1,relu,relu,elu,selu,NaN,NaN,layers.Dropout,layers.GaussianDropout,58,66,104,46,214,0,2,2,2,3,0.066515,0.436444,0.214327,0.482422,"(50, 4)","(128,)",4.0,0.0,128.0,1.0,587905.0,24.862507,0.597488
2,1,2,layers.MaxPooling1D,layers.LocallyConnected1D,2,81,2,3,1,3,layers.MaxPooling1D,layers.SeparableConv1D,3,82,layers.SimpleRNN,1,1,optimizers.SGD(),0,relu,hard_sigmoid,elu,relu,layers.GaussianNoise,NaN,layers.AlphaDropout,layers.GaussianNoise,180,32,96,45,160,0,3,3,2,3,0.473070,0.269308,0.067590,0.599020,"(64, 50)","(128,)",64.0,1.0,128.0,0.0,2593520.0,128.819910,0.596564
0,1,4,layers.MaxPooling1D,layers.LocallyConnected1D,0,207,0,0,4,1,NaN,layers.Conv1D,1,61,layers.LSTM,1,0,optimizers.RMSprop(),0,relu,softplus,selu,exponential,layers.GaussianNoise,layers.AlphaDropout,layers.GaussianNoise,layers.GaussianDropout,184,41,150,32,206,1,0,1,1,3,0.566049,0.608037,0.542902,0.427441,"(50, 8)","(128,)",8.0,1.0,128.0,1.0,504339.0,17.733969,0.596287
0,1,2,NaN,layers.LocallyConnected1D,3,71,3,3,1,2,NaN,layers.LocallyConnected1D,2,148,layers.LSTM,1,1,optimizers.SGD(),0,exponential,sigmoid,elu,tanh,NaN,layers.GaussianDropout,NaN,NaN,174,212,74,19,80,1,1,3,3,0,0.313049,0.553642,0.756240,0.003206,"(50, 32)","(16,)",32.0,1.0,16.0,1.0,5868171.0,540.512901,0.595363
1,4,4,layers.MaxPooling1D,layers.SeparableConv1D,3,192,0,0,4,1,NaN,layers.Conv1D,2,117,layers.GRU,1,0,optimizers.Adagrad(),1,sigmoid,softsign,sigmoid,softplus,layers.GaussianDropout,layers.AlphaDropout,layers.AlphaDropout,layers.AlphaDropout,215,189,224,42,46,0,2,2,3,3,0.723191,0.451688,0.172363,0.260639,"(50, 4)","(482,)",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,layers.MaxPooling1D,layers.LocallyConnected1D,0,230,2,2,9,2,layers.MaxPooling1D,layers.SeparableConv1D,3,18,layers.SimpleRNN,1,0,optimizers.Adagrad(),1,hard_sigmoid,softsign,sigmoid,elu,layers.AlphaDropout,layers.AlphaDropout,NaN,NaN,197,221,220,188,22,0,0,1,1,0,0.360941,0.697102,0.025490,0.368324,"(50, 64)","(16,)",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,1,layers.MaxPooling1D,layers.LocallyConnected1D,0,201,3,0,5,4,NaN,layers.Conv1D,0,49,layers.LSTM,1,0,optimizers.Adamax(),1,selu,softsign,selu,sigmoid,layers.Dropout,NaN,layers.Dropout,layers.GaussianNoise,70,231,101,134,108,1,1,0,0,0,0.371397,0.350227,0.558116,0.538811,"(50, 16)","(482,)",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,7,1,NaN,layers.SeparableConv1D,1,39,3,2,6,3,NaN,layers.Conv1D,2,188,layers.GRU,1,0,optimizers.SGD(),1,softsign,tanh,sigmoid,elu,NaN,layers.GaussianDropout,layers.GaussianDropout,layers.GaussianNoise,111,109,100

In [14]:

df = df.fillna(-1)
options = dict()
encoders = dict()

for i in df.columns:
    if df[i].dtype == 'object':
        encoder = preprocessing.LabelEncoder()
        df[i] = encoder.fit_transform(df[i].astype(str))
        encoders[i] = encoder
    options[i] = list(set(df[i]))
        

In [15]:
df.head()

,convolutional_kernel_size_1,convolutional_pool_size_1,pooling_algorithm_1,convolution_type_1,convolutional_layers_1,convolutional_filters_1,conv_layers_per_pooling_layer_1,conv_layers_per_pooling_layer_2,convolutional_kernel_size_2,convolutional_pool_size_2,pooling_algorithm_2,convolution_type_2,convolutional_layers_2,convolutional_filters_2,recurrent_type,use_x1,use_x2,optimizer_algorithm,batchnorm,block_1_activations,block_2_activations,merge_activations,top_block_activations,block_1_dropout,block_2_dropout,merge_block_dropout,top_block_dropout,merge_block_width,block_2_width,top_block_width,recurrent_layers_width,block_1_dense_layers_width,merge_block_layers,block_2_layers,dense_top_layers,recurrent_layers,block_1_dense_layers,block_1_dropout_amount,block_2_dropout_amount,merge_block_dropout_amount,top_block_dropout_amount,input_shape1,input_shape2,history_length,timeseries_scaled,general_feature_encoding_size,rotate_history_data,num_of_params,model_training_time,accuracy
0,9,1,0,2,2,62,2,2,9,3,1,1,1,59,1,1,1,3,1,6,6,5,1,1,4,4,4,92,88,78,52,44,0,0,2,2,0,0.236232,0.225616,0.017484,0.273730,9,2,-1.0,0,-1.0,-1.0,-1.0,-1.000000,-1.000000
1,2,2,1,0,2,16,0,1,4,3,0,2,1,120,1,0,1,3,1,2,6,7,5,3,2,1,0,85,66,113,70,73,0,0,0,2,0,0.742149,0.636627,0.086401,0.089708,4,1,32.0,2,16.0,1.0,1453.0,1.957229,0.616370
2,6,2,0,1,2,62,0,2,8,1,1,2,1,25,0,0,1,0,1,0,4,0,6,0,4,0,2,123,59,67,87,24,1,0,1,1,0,0.591238,0.123675,0.099515,0.667789,7,2,8.0,2,-1.0,1.0,207933.0,6.268083,0.624477
3,6,1,0,1,1,49,0,0,7,4,0,2,0,124,0,0,1,1,1,1,4,8,8,1,0,2,0,24,125,108,32,27,0,0,0,2,1,0.571204,0.599452,0.536833,0.039133,5,2,4.0,2,-1.0,1.0,432001.0,5.557616,0.642259
4,9,3,0,1,1,68,0,2,7,4,1,2,1,19,1,0,1,4,1,7,5,8,1,0,4,0,2,24,73,113,63,23,0,1,2,1,1,0.690138,0.645096,0.589822,0.169776,8,1,64.0,2,16.0,0.0,23343.0,2.977515,0.610879


In [6]:

# rf = ensemble.RandomForestRegressor()
model = linear_model.ElasticNet()
x = df.drop('accuracy', axis = 1)

x2 = x.copy()
for i1 in x.columns:
    for i2 in x.columns:
        x2[f'prod_of_{i1}_{i2}'] = x2[i1] * x2[i2]
y = df['accuracy']

model.fit(x, y)


ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [7]:

recs = []
for n in range(1000):
    rec = []
    for i in x.columns:
        rec.append(random.choice(options[i]))
    recs.append(rec)
    

In [9]:

df_rec = pd.DataFrame(data = recs,
                     columns = x.columns)
df_rec.head()

,convolutional_kernel_size,convolutional_pool_size,pooling_algorithm,convolution_type,recurrent_type,use_x1,use_x2,optimizer_algorithm,conv_layers_per_pooling_layer,batchnorm,block_1_activations,block_2_activations,merge_activations,top_block_activations,block_1_dropout,block_2_dropout,merge_block_dropout,top_block_dropout,merge_block_width,block_2_width,top_block_width,recurrent_layers_width,block_1_dense_layers_width,merge_block_layers,block_2_layers,convolutional_filters,dense_top_layers,convolutional_layers,recurrent_layers,block_1_dense_layers,block_1_dropout_amount,block_2_dropout_amount,merge_block_dropout_amount,top_block_dropout_amount,input_shape1,input_shape2,history_length,timeseries_scaled,general_feature_encoding_size,rotate_history_data,num_of_params,model_training_time
0,8,3,layers.MaxPooling1D,layers.SeparableConv1D,layers.SimpleRNN,0,1,optimizers.Nadam(),1,1,softplus,softsign,sigmoid,selu,-1,-1,layers.AlphaDropout,layers.GaussianNoise,164,203,146,19,49,1,2,23,1,1,0,2,0.773785,0.569988,0.018958,0.018909,"(81, 8)","(16,)",4.0,-1,16.0,-1.0,209441.0,2.691719
1,1,3,-1,layers.SeparableConv1D,layers.SimpleRNN,1,0,optimizers.Adagrad(),0,0,tanh,sigmoid,tanh,selu,layers.AlphaDropout,layers.Dropout,layers.GaussianDropout,-1,65,166,217,144,82,0,2,66,0,0,1,1,0.183318,0.607368,0.323049,0.679372,"(81, 64)","(16,)",16.0,False,32.0,1.0,139065.0,2.689170
2,5,3,layers.MaxPooling1D,layers.LocallyConnected1D,layers.GRU,1,1,optimizers.Adamax(),2,0,elu,softsign,elu,hard_sigmoid,layers.AlphaDropout,layers.GaussianDropout,layers.AlphaDropout,layers.Dropout,62,52,53,145,229,0,2,27,2,0,1,1,0.377385,0.487088,0.587144,0.760042,"(81, 32)","(64,)",64.0,True,-1.0,1.0,393187.0,2.261715
3,3,4,layers.MaxPooling1D,layers.SeparableConv1D,layers.GRU,1,1,optimizers.Adamax(),2,0,elu,hard_sigmoid,hard_sigmoid,tanh,layers.Dropout,-1,layers.GaussianNoise,-1,16,238,152,163,59,1,1,84,1,2,2,0,0.183318,0.449211,0.576362,0.271564,"(81, 8)","(16,)",-1.0,True,4.0,-1.0,1057945.0,6.383539
4,8,1,layers.MaxPooling1D,layers.Conv1D,layers.SimpleRNN,0,1,optimizers.RMSprop(),1,0,sigmoid,sigmoid,sigmoid,selu,layers.Dropout,layers.AlphaDropout,layers.Dropout,layers.GaussianDropout,193,207,164,124,16,1,2,124,0,2,0,0,0.759307,0.369651,0.061024,0.497883,"(4, 81)","(2500,)",4.0,False,32.0,-1.0,3727806.0,19.225342


In [8]:


df_rec_copy = df_rec.copy()

for i in encoders.keys():
    df_rec_copy[i] = encoders[i].transform(df_rec_copy[i])

df_rec['accuracy'] = model.predict(df_rec_copy)

df_rec.sort_values('accuracy', ascending = False).head(20)


ValueError: y contains previously unseen labels: -1